In [53]:
import skfem
import meshio
import numpy as np
import matplotlib.pyplot as plt
import h5py

import pandas as pd
from skfem import (
    MeshTri, Basis, FacetBasis, ElementTriP2, ElementTriP1, ElementVector,
    asm, bmat, condense, solve, BilinearForm, LinearForm, MeshTet, ElementTetP2, ElementTetP1
)
from skfem.models.general import divergence
from skfem.models.poisson import vector_laplace, mass
from skfem.helpers import grad, dot, laplacian

import scipy.sparse as sp
from scipy.sparse.linalg import eigsh, eigs, splu, LinearOperator

import scipy.linalg
import pandas as pd
import plotly.graph_objects as go



In [2]:
f = h5py.File('Carotid3D_h1.h5', 'r')
print(list(f.keys()))

['boundaries', 'mesh']


In [4]:
coordinates_mesh = f['mesh/coordinates'][:]
elements_mesh    = f['mesh/topology'][:]

coordinates_boundaries = f['/boundaries/coordinates'][:]
elements_boundaries    = f['/boundaries/topology'][:]
values_boundaries      = f['/boundaries/values'][:]

In [26]:
# malla = meshio.Mesh(
#     points=coordinates_mesh,
#     cells={"tetra": elements_mesh})

In [25]:
np.where(f['/boundaries/values'][:] == 0)[0].shape

(984801,)

In [50]:
boundary_1 = np.where(f['/boundaries/values'][:] == 1)[0]
boundary_elements_1 = elements_boundaries[boundary_1]

boundary_2 = np.where(f['/boundaries/values'][:] == 2)[0]
boundary_elements_2 = elements_boundaries[boundary_2]

boundary_3 = np.where(f['/boundaries/values'][:] == 3)[0]
boundary_elements_3 = elements_boundaries[boundary_3]

boundary_4 = np.where(f['/boundaries/values'][:] == 4)[0]
boundary_elements_4 = elements_boundaries[boundary_4]

boundary_5 = np.where(f['/boundaries/values'][:] == 5)[0]
boundary_elements_5 = elements_boundaries[boundary_5]

boundary_6 = np.where(f['/boundaries/values'][:] == 6)[0]
boundary_elements_6 = elements_boundaries[boundary_6]

boundary_7 = np.where(f['/boundaries/values'][:] == 7)[0]
boundary_elements_7 = elements_boundaries[boundary_7]

boundary_8 = np.where(f['/boundaries/values'][:] == 8)[0]
boundary_elements_8 = elements_boundaries[boundary_8]

boundaries = [boundary_elements_1, boundary_elements_2,
              boundary_elements_3, boundary_elements_4,
              boundary_elements_5, boundary_elements_6,
              boundary_elements_7, boundary_elements_8]

boundary_wall = boundary_1
boundary_inflow = boundary_2
boundary_outflow = np.concatenate([boundary_3, boundary_4, boundary_5, boundary_6, boundary_7, boundary_8])

In [51]:
mesh = MeshTet(coordinates_mesh.T, elements_mesh.T)

Transforming over 1000 vertices to C_CONTIGUOUS.
Transforming over 1000 elements to C_CONTIGUOUS.


In [57]:
# # Graficar malla y boundary facets
# facets = mesh.facets
# nodes  = mesh.p


# def get_mesh_data(indices_facets):  
#     if indices_facets.size == 0:
#         return None
        
#     # Extraer las facetas que corresponden a los índices calculados
#     facet_indices = facets[:, indices_facets]
    
#     x = nodes[0, :]
#     y = nodes[1, :]
#     z = nodes[2, :]
    
#     i = facet_indices[0, :]
#     j = facet_indices[1, :]
#     k = facet_indices[2, :]
    
#     return x, y, z, i, j, k


# traces = []

# # Wall
# wall_data = get_mesh_data(boundary_wall)
# if wall_data:
#     x, y, z, i, j, k = wall_data
#     traces.append(
#         go.Mesh3d(
#             x=x, y=y, z=z, i=i, j=j, k=k,
#             color='rgba(0, 200, 0, 0.5)',
#             opacity=0.5,
#             name='Wall',
#             showlegend=True,
#             lighting=dict(ambient=0.5, diffuse=0.5, specular=0.8)
#         )
#     )

# # # Inflow
# # inflow_data = get_mesh_data(inflow)
# # if inflow_data:
# #     x, y, z, i, j, k = inflow_data
# #     traces.append(
# #         go.Mesh3d(
# #             x=x, y=y, z=z, i=i, j=j, k=k,
# #             color='rgba(200, 0, 0, 0.8)',
# #             opacity=0.8,
# #             name='Inflow',
# #             showlegend=True,
# #             lighting=dict(ambient=0.5, diffuse=0.5, specular=0.8)
# #         )
# #     )

# # Outflow
# outflow_data = get_mesh_data(boundary_2)
# if outflow_data:
#     x, y, z, i, j, k = outflow_data
#     traces.append(
#         go.Mesh3d(
#             x=x, y=y, z=z, i=i, j=j, k=k,
#             color='rgba(0, 0, 200, 0.8)',
#             opacity=0.8,
#             name='Outflow',
#             showlegend=True,
#             lighting=dict(ambient=0.5, diffuse=0.5, specular=0.8)
#         )
#     )

# # Nodos de la malla
# traces.append(
#     go.Scatter3d(
#         x=nodes[0, :],
#         y=nodes[1, :],
#         z=nodes[2, :],
#         mode='markers',
#         marker=dict(
#             size=1,
#             color=nodes[2, :],
#             opacity=0.8
#         ),
#         name='Nodos Malla',
#         showlegend=True
#     )
# )

# fig = go.Figure(data=traces)

# fig.update_layout(
#     title='Malla 3D Interactiva de Fronteras de Tubería en U',
#     scene=dict(
#         xaxis_title='X',
#         yaxis_title='Y',
#         zaxis_title='Z',
#         aspectmode='data'
#     ),
#     showlegend=True,
#     legend=dict(
#         yanchor="top",
#         y=0.99,
#         xanchor="left",
#         x=0.01
#     ))

# fig.show()


In [54]:
# Definir elementos y bases (P2 para velocidad, P1 para presión) en 3D
element = {
    'u': ElementVector(ElementTetP2(), dim=3),
    'p': ElementTetP1(),
}
basis = {
    'u': Basis(mesh, element['u'], intorder=4),
    'p': Basis(mesh, element['p'], intorder=4),
}
basis_u, basis_p = basis['u'], basis['p']
Nu, Np = basis_u.N, basis_p.N
N      = Nu + Np

In [55]:
mu    = 0.035 # viscosidad dinámica
rho   = 1.0   # densidad
nu    = mu / rho # Viscosidad cinemática
V_car = 1.0    # Velocidad característica
L_car = 1.0    # Longitud característica
Re    = V_car * L_car / nu


In [56]:
@BilinearForm
def mass_matrix(u, v, w):
    return dot(u, v)

# Ensamblaje de matrices
A =  asm(vector_laplace, basis_u)               
B = -asm(divergence, basis_u, basis_p)   

F_u = basis['u'].zeros()
F_p = basis['p'].zeros()
F = np.hstack([F_u, F_p])

KeyboardInterrupt: 

In [ ]:
dofs_entrada  = basis_u.get_dofs(boundary_inflow).all()
dofs_p_salida = basis_p.get_dofs(boundary_outflow).all()

xin   = basis_u.doflocs[0, dofs_entrada[::2]]
yin   = basis_u.doflocs[1, dofs_entrada[1::2]]

def u_in_y(x, y):
    return  (y-np.max(yin))*(np.min(yin)-y) + x*0.0

x_boundaries = np.zeros(Nu+Np)
x_boundaries[dofs_entrada[::2]]  = u_in_y(xin, yin)/np.max(u_in_y(xin, yin))
x_boundaries[dofs_entrada[1::2]] = 0.0

x_boundaries[Nu + dofs_p_salida[0]] = 0.0

D_all = np.concatenate([
    basis_u.get_dofs(boundary_wall).all(),
    basis_u.get_dofs(boundary_inflow).all(),
    Nu+np.array([dofs_p_salida[0]])
]) 